In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 13.3 MB/s 
     |████████████████████████████████| 168 kB 61.9 MB/s 
     |████████████████████████████████| 182 kB 76.5 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 168 kB 77.5 MB/s 
     |████████████████████████████████| 166 kB 78.5 MB/s 
     |████████████████████████████████| 166 kB 82.1 MB/s 
     |████████████████████████████████| 162 kB 81.7 MB/s 
     |████████████████████████████████| 162 kB 77.1 MB/s 
     |████████████████████████████████| 158 kB 84.4 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 85.5 MB/s 
     |████████████████████████████████| 157 kB 85.0 MB/s 
     |████████████████████████████████| 157 kB 87.5 MB/s 
     |████████████████████████████████| 157 kB 85.0 MB/s 
     |████████████████████████████████| 157 kB 82.3 MB/s 
     |████████████████████████████████| 157 kB 83.5 MB/s 
     |██████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [19]:
%env WANDB_NOTEBOOK_NAME = "ResNet152"

env: WANDB_NOTEBOOK_NAME="ResNet152"


In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.


In [6]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [20]:
tf.keras.backend.clear_session()
resnet152 = tf.keras.applications.ResNet152(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling='avg')
model_resnet152=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet152,
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet152.trainable=False
model_resnet152.summary()

234698864/234698864 [==============================] - 11s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet152 (Functional)      (None, 2048)              58370944  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 58,381,189
Trainable params: 10,245
Non-trainable params: 58,370,944
_________________________________________________________________


In [28]:
model_resnet152.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'],
            #   jit_compile = True
              )

In [29]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet152",
)

In [30]:
%%time
epochs=40
history = model_resnet152.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback(
    compute_flops = True,
    save_model = True
)]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.4670 - accuracy: 0.3949

wandb: Adding directory to artifact (/content/wandb/run-20221208_101606-1law5ujp/files/model-best)... Done. 1.6s


22/22 [==============================] - 80s 3s/step - loss: 1.4670 - accuracy: 0.3949 - val_loss: 1.3708 - val_accuracy: 0.3886
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.3413 - accuracy: 0.4560

wandb: Adding directory to artifact (/content/wandb/run-20221208_101606-1law5ujp/files/model-best)... Done. 1.4s


22/22 [==============================] - 69s 3s/step - loss: 1.3413 - accuracy: 0.4560 - val_loss: 1.2853 - val_accuracy: 0.5143
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 1.2772 - accuracy: 0.5099

...

22/22 [==============================] - 71s 3s/step - loss: 1.0030 - accuracy: 0.6321 - val_loss: 1.0354 - val_accuracy: 0.5657
Epoch 37/40
22/22 [==============================] - ETA: 0s - loss: 0.9907 - accuracy: 0.6193

wandb: Adding directory to artifact (/content/wandb/run-20221208_101606-1law5ujp/files/model-best)... Done. 1.4s


22/22 [==============================] - 70s 3s/step - loss: 0.9907 - accuracy: 0.6193 - val_loss: 1.0229 - val_accuracy: 0.5943
Epoch 38/40
22/22 [==============================] - 9s 382ms/step - loss: 0.9876 - accuracy: 0.6406 - val_loss: 1.0281 - val_accuracy: 0.5657
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.9760 - accuracy: 0.6222

wandb: Adding directory to artifact (/content/wandb/run-20221208_101606-1law5ujp/files/model-best)... Done. 1.6s


22/22 [==============================] - 71s 3s/step - loss: 0.9760 - accuracy: 0.6222 - val_loss: 1.0155 - val_accuracy: 0.6114
Epoch 40/40
22/22 [==============================] - 9s 383ms/step - loss: 0.9878 - accuracy: 0.6392 - val_loss: 1.0305 - val_accuracy: 0.5657
CPU times: user 26min 38s, sys: 1min 31s, total: 28min 10s
Wall time: 34min 23s


In [31]:
wandb.alert(title="Run finished",
            text='ResNet152')

In [32]:
wandb.finish()

accuracy,▁▃▄▅▄▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇▇█▇█▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▆▅▆▅▅▅▆▆▅▆▆▇▇▆▆▆▆██▆▇▆█▆
val_loss,█▆▆▅▅▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
GFLOPs,11.29242
accuracy,0.6392
best_epoch,38
best_val_loss,1.01552
epoch,39
loss,0.98776


In [18]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
